In [1]:
import os 
import pandas as pd
import PIL
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from collections import Counter
from skimage import io
import tensorflow as tf
from importlib import reload
import hashlib
from collections import defaultdict
import shutil
import numpy as np
from tensorflow.keras import layers, models
import utils as ut

2025-11-12 18:27:59.557955: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
train_path = "/Users/chloelsmith/Desktop/archive-2/Grayscale_Training"
test_path = "/Users/chloelsmith/Desktop/archive-2/Grayscale_Testing"


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import models
from tensorflow.keras.preprocessing import image_dataset_from_directory
from cleanlab.outlier import OutOfDistribution
from cleanlab.rank import find_top_issues

# -----------------------------
# USER CONFIG
# -----------------------------
train_path = "/Users/chloelsmith/Desktop/archive-2/Grayscale_Training"
test_path = "/Users/chloelsmith/Desktop/archive-2/Grayscale_Testing"

img_size = (224, 224)
batch_size = 64  # Larger batch speeds up CPU runs

# -----------------------------
# LOAD DATASETS (grayscale -> RGB)
# -----------------------------
def to_rgb(image, label):
    image = tf.image.grayscale_to_rgb(image)             # Convert 1 channel → 3
    image = tf.image.convert_image_dtype(image, tf.float32)  # Normalize to [0,1]
    return image, label

train_ds = image_dataset_from_directory(
    train_path,
    image_size=img_size,
    color_mode='grayscale',
    batch_size=batch_size,
    shuffle=False
).map(to_rgb, num_parallel_calls=tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE)

test_ds = image_dataset_from_directory(
    test_path,
    image_size=img_size,
    color_mode='grayscale',
    batch_size=batch_size,
    shuffle=False
).map(to_rgb, num_parallel_calls=tf.data.AUTOTUNE).cache().prefetch(tf.data.AUTOTUNE)

# -----------------------------
# MODEL: Pretrained ResNet50
# -----------------------------
base_model = ResNet50(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    input_shape=(*img_size, 3)
)
model = models.Model(inputs=base_model.input, outputs=base_model.output)
model.trainable = False  # evaluation mode

# -----------------------------
# EMBEDDING FUNCTION
# -----------------------------
def embed_images(model, dataset, save_path=None):
    feature_embeddings = []
    for batch_images, _ in dataset:
        embeddings = model(batch_images, training=False)
        feature_embeddings.append(embeddings.numpy())
    feature_embeddings = np.concatenate(feature_embeddings, axis=0)
    
    if save_path:
        np.save(save_path, feature_embeddings)
        print(f"Saved embeddings to {save_path}")
        
    return feature_embeddings

# -----------------------------
# GENERATE EMBEDDINGS
# -----------------------------
print("Generating embeddings for train set...")
train_feature_embeddings = embed_images(model, train_ds, save_path="train_embeddings.npy")
print(f"Train embeddings shape: {train_feature_embeddings.shape}")

print("Generating embeddings for test set...")
test_feature_embeddings = embed_images(model, test_ds, save_path="test_embeddings.npy")
print(f"Test embeddings shape: {test_feature_embeddings.shape}")

# -----------------------------
# CLEANLAB OOD DETECTION
# -----------------------------
ood = OutOfDistribution()
train_ood_scores = ood.fit_score(features=train_feature_embeddings)

# Find top outliers
top_train_ood_idxs = find_top_issues(quality_scores=train_ood_scores, topk=15)
print("Top possible OOD samples in training set:")
print(top_train_ood_idxs)


2025-11-12 18:46:06.047118: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/chloelsmith/anaconda3/envs/washu/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Found 5521 files belonging to 4 classes.
Found 1205 files belonging to 4 classes.
Generating embeddings for train set...


2025-11-12 18:52:51.300194: W tensorflow/core/kernels/data/cache_dataset_ops.cc:858] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [ ]:
visualize_outliers(top_train_ood_features_idxs, train_data)